In [1]:
from __future__ import print_function
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop, Adam, SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.datasets import mnist
from keras.utils import np_utils

import pandas as pd
import numpy as np

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek

In [3]:
def gini(solution, submission):
    df = zip(solution, submission, range(len(solution)))
    df = sorted(df, key=lambda x: (x[1],-x[2]), reverse=True)
    rand = [float(i+1)/float(len(df)) for i in range(len(df))]
    totalPos = float(sum([x[0] for x in df]))
    cumPosFound = [df[0][0]]
    for i in range(1,len(df)):
        cumPosFound.append(cumPosFound[len(cumPosFound)-1] + df[i][0])
    Lorentz = [float(x)/totalPos for x in cumPosFound]
    Gini = [Lorentz[i]-rand[i] for i in range(len(df))]
    return sum(Gini)

def normalized_gini(solution, submission):
    normalized_gini = gini(solution, submission)/gini(solution, solution)
    return normalized_gini

In [4]:
from sklearn.metrics import f1_score, roc_auc_score
import keras.callbacks as kc

class Metrics(kc.Callback):
    def on_epoch_end(self, batch, logs={}):
        predict = np.asarray(self.model.predict(self.validation_data[0]))
        targ = self.validation_data[1]
        self.ginis=(2*roc_auc_score(targ, predict))-1
        return
metrics = Metrics()

In [39]:
def data():
    non_imp = ['ps_ind_12_bin','ps_ind_13_bin','ps_ind_18_bin','ps_car_10_cat','ps_ind_11_bin','ps_ind_10_bin','ps_ind_14']
    
    train = (pd.read_csv('../data/train.csv', na_values=999)
              .fillna(value=999))
    unwanted = list((set(train.columns[train.columns.str.startswith('ps_calc_')])|set(non_imp)))
    train.drop(unwanted, axis=1, inplace=True)
            
    test  = (pd.read_csv('../data/test.csv', na_values=999)
              .fillna(value=999)
              .drop(unwanted, axis=1)) 
            
    X = train.drop(['id', 'target'], axis=1).values
    y = train.target.values
    test_id = test.id.values
    test = test.drop('id', axis=1)

    
    f_dicts ={}

    features_dict_data_type = {}
    for k in ['cat', 'bin', 'con']:
        features_dict_data_type[k]=[]
        for i in train.columns.tolist()[2:]:
            j = i.split('_')
            if len(j) == 3:
                j.append('con')
            if j[3]==k:
                features_dict_data_type[k].append(i)
    f_dicts['type'] = features_dict_data_type

    train_cat = np.array(train[f_dicts['type']['cat']])
    train_cat[:,:] = np.add(train_cat[:,:],np.ones((train_cat.shape[0],train_cat.shape[1])))
    test_cat  = np.array(test[f_dicts['type']['cat']])
    test_cat[:,:] =  np.add(test_cat[:,:],np.ones((test_cat.shape[0],train_cat.shape[1])))

    OH = OneHotEncoder()
    OH.fit(np.array(list(train_cat) + list(test_cat)))
    train_cat = OH.transform(train_cat).toarray()
    test_cat = OH.transform(test_cat).toarray()

    train_con = np.array(train[f_dicts['type']['con']])
    test_con  = np.array(test[f_dicts['type']['con']])
    RS = StandardScaler()
    RS.fit(list(train_con) + list(test_con))
    train_con = RS.transform(train_con)
    train_con = RS.transform(train_con)

    train_bin = np.array(train[f_dicts['type']['bin']])
    test_bin = np.array(test[f_dicts['type']['bin']])


    X = np.hstack((train_cat,train_con,train_bin))

    X_test = np.hstack((test_cat,test_con,test_bin))
    
    #smote = SMOTE(random_state=0)
    #X_resampled, y_resampled = smote.fit_sample(X, y)
    X_train, X_vali, y_train, y_vali = train_test_split(X, y, test_size=0.25, shuffle=True,  random_state=np.random.randint(1000), stratify=y)

    return X_train, X_vali, y_train, y_vali, X_test, test_id

In [13]:
class Metrics(kc.Callback):
        def on_epoch_end(self, batch, logs={}):
            predict = np.asarray(self.model.predict(self.validation_data[0]))
            targ = self.validation_data[1]
            self.ginis=(2*roc_auc_score(targ, predict))-1
            return

In [7]:
def model():
    class Metrics(kc.Callback):
        def on_epoch_end(self, batch, logs={}):
            predict = np.asarray(self.model.predict(self.validation_data[0]))
            targ = self.validation_data[1]
            self.ginis=(2*roc_auc_score(targ, predict))-1
            return
    metrics = Metrics()
    model = Sequential()
    model.add(Dense({{choice([128, 256, 512])}}, input_dim=198, kernel_initializer='glorot_uniform',bias_initializer='zeros', activation={{choice(['relu', 'tanh', 'sigmoid'])}}))#{{choice(['relu', 'tanh', 'linear'])}}
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([128, 256, 512])}}, kernel_initializer='glorot_uniform',bias_initializer='zeros', activation={{choice(['relu', 'tanh', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([128, 256, 512, 1024])}}, kernel_initializer='glorot_uniform',bias_initializer='zeros', activation={{choice(['relu', 'tanh', 'sigmoid'])}}))
    model.add(Dense(1,activation={{choice(['tanh', 'sigmoid'])}}, kernel_initializer='glorot_uniform'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['acc'])
    
    ES = EarlyStopping(monitor='val_loss', min_delta=0, patience=1, verbose=1, mode='auto')
    model.fit(X_train, y_train, batch_size={{choice([64, 128])}}, epochs= 20,validation_data=(X_vali,y_vali), callbacks=[ES,metrics])#, class_weight = class_weight
    score, acc = model.evaluate(X_vali, y_vali, verbose=0)
    print('Test accuracy: ', metrics.ginis)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [47]:
X_train, X_vali, y_train, y_vali, X_test, test_id = data()

In [9]:
best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=50,
                                      trials=Trials(),
                                      notebook_name='dlsearch')

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Input
except:
    pass

try:
    from keras.layers.core import Dense, Dropout, Activation
except:
    pass

try:
    from keras.optimizers import RMSprop, Adam, SGD
except:
    pass

try:
    from keras.callbacks import EarlyStopping, ModelCheckpoint
except:
    pass

try:
    from keras.datasets import mnist
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from keras.wrappers.scikit_learn import KerasRegressor
except:
    pass

try:
    from sklearn.model_selection import cross_val_score, train_tes

860277/860277 [==============================] - 37s - loss: 0.4361 - acc: 0.7956 - val_loss: 0.3139 - val_acc: 0.8788
Epoch 3/20
860277/860277 [==============================] - 37s - loss: 0.3657 - acc: 0.8378 - val_loss: 0.2635 - val_acc: 0.9000
Epoch 4/20
860277/860277 [==============================] - 37s - loss: 0.3311 - acc: 0.8575 - val_loss: 0.2489 - val_acc: 0.9049
Epoch 5/20
860277/860277 [==============================] - 37s - loss: 0.3088 - acc: 0.8698 - val_loss: 0.2557 - val_acc: 0.8978
Epoch 6/20
860277/860277 [==============================] - 37s - loss: 0.2940 - acc: 0.8777 - val_loss: 0.2213 - val_acc: 0.9183
Epoch 7/20
860277/860277 [==============================] - 37s - loss: 0.2821 - acc: 0.8843 - val_loss: 0.2184 - val_acc: 0.9190
Epoch 8/20
860277/860277 [==============================] - 37s - loss: 0.2729 - acc: 0.8888 - val_loss: 0.2077 - val_acc: 0.9269
Epoch 9/20
860277/860277 [==============================] - 37s - loss: 0.2650 - acc: 0.8932 - val_lo

860277/860277 [==============================] - 24s - loss: 0.5771 - acc: 0.6729 - val_loss: 0.5058 - val_acc: 0.7412
Epoch 3/20
860277/860277 [==============================] - 23s - loss: 0.4753 - acc: 0.7607 - val_loss: 0.3865 - val_acc: 0.8233
Epoch 4/20
860277/860277 [==============================] - 23s - loss: 0.4009 - acc: 0.8124 - val_loss: 0.3275 - val_acc: 0.8618
Epoch 5/20
860277/860277 [==============================] - 23s - loss: 0.3601 - acc: 0.8382 - val_loss: 0.3094 - val_acc: 0.8685
Epoch 6/20
860277/860277 [==============================] - 23s - loss: 0.3341 - acc: 0.8533 - val_loss: 0.2769 - val_acc: 0.8842
Epoch 7/20
860277/860277 [==============================] - 23s - loss: 0.3147 - acc: 0.8659 - val_loss: 0.2696 - val_acc: 0.8885
Epoch 8/20
860277/860277 [==============================] - 23s - loss: 0.3004 - acc: 0.8749 - val_loss: 0.2575 - val_acc: 0.8979
Epoch 9/20
860277/860277 [==============================] - 23s - loss: 0.2891 - acc: 0.8815 - val_lo

860277/860277 [==============================] - 40s - loss: 0.1919 - acc: 0.9277 - val_loss: 0.1692 - val_acc: 0.9384
Epoch 14/20
860277/860277 [==============================] - 39s - loss: 0.1887 - acc: 0.9288 - val_loss: 0.1708 - val_acc: 0.9375
Epoch 15/20
860277/860277 [==============================] - 39s - loss: 0.1844 - acc: 0.9308 - val_loss: 0.1569 - val_acc: 0.9458
Epoch 16/20
860277/860277 [==============================] - 40s - loss: 0.1811 - acc: 0.9323 - val_loss: 0.1622 - val_acc: 0.9416
Epoch 17/20
860277/860277 [==============================] - 40s - loss: 0.1783 - acc: 0.9337 - val_loss: 0.1630 - val_acc: 0.9417
Epoch 00016: early stopping
Test accuracy:  0.953302401761
Train on 860277 samples, validate on 286759 samples
Epoch 1/20
860277/860277 [==============================] - 41s - loss: 0.6351 - acc: 0.6196 - val_loss: 0.5267 - val_acc: 0.7240
Epoch 2/20
860277/860277 [==============================] - 40s - loss: 0.4356 - acc: 0.7869 - val_loss: 0.3509 - va

860277/860277 [==============================] - 43s - loss: 0.2096 - acc: 0.9196 - val_loss: 0.1836 - val_acc: 0.9311
Epoch 19/20
860277/860277 [==============================] - 43s - loss: 0.2067 - acc: 0.9212 - val_loss: 0.1737 - val_acc: 0.9384
Epoch 20/20
860277/860277 [==============================] - 44s - loss: 0.2046 - acc: 0.9223 - val_loss: 0.1742 - val_acc: 0.9382
Test accuracy:  0.945912075544
Train on 860277 samples, validate on 286759 samples
Epoch 1/20
860277/860277 [==============================] - 44s - loss: 0.6777 - acc: 0.5789 - val_loss: 0.6582 - val_acc: 0.6064
Epoch 2/20
860277/860277 [==============================] - 43s - loss: 0.5849 - acc: 0.6846 - val_loss: 0.4452 - val_acc: 0.8074
Epoch 3/20
860277/860277 [==============================] - 43s - loss: 0.4367 - acc: 0.7955 - val_loss: 0.4122 - val_acc: 0.8087
Epoch 4/20
860277/860277 [==============================] - 43s - loss: 0.3657 - acc: 0.8383 - val_loss: 0.3647 - val_acc: 0.8360
Epoch 5/20
86027

860277/860277 [==============================] - 44s - loss: 0.2294 - acc: 0.9119 - val_loss: 0.2873 - val_acc: 0.9166
Epoch 00010: early stopping
Test accuracy:  0.951870175287
Train on 860277 samples, validate on 286759 samples
Epoch 1/20
860277/860277 [==============================] - 46s - loss: 0.6868 - acc: 0.5539 - val_loss: 0.6787 - val_acc: 0.5853
Epoch 2/20
860277/860277 [==============================] - 44s - loss: 0.6761 - acc: 0.5790 - val_loss: 0.6720 - val_acc: 0.5902
Epoch 3/20
860277/860277 [==============================] - 44s - loss: 0.6727 - acc: 0.5845 - val_loss: 0.6686 - val_acc: 0.5964
Epoch 4/20
860277/860277 [==============================] - 44s - loss: 0.6688 - acc: 0.5907 - val_loss: 0.6622 - val_acc: 0.6086
Epoch 5/20
860277/860277 [==============================] - 44s - loss: 0.6636 - acc: 0.5985 - val_loss: 0.6608 - val_acc: 0.6267
Epoch 6/20
860277/860277 [==============================] - 44s - loss: 0.6576 - acc: 0.6083 - val_loss: 0.6534 - val_ac

860277/860277 [==============================] - 45s - loss: 0.3187 - acc: 0.8617 - val_loss: 0.2741 - val_acc: 0.8844
Epoch 7/20
860277/860277 [==============================] - 45s - loss: 0.3048 - acc: 0.8697 - val_loss: 0.2696 - val_acc: 0.8848
Epoch 8/20
860277/860277 [==============================] - 45s - loss: 0.2959 - acc: 0.8749 - val_loss: 0.2651 - val_acc: 0.8896
Epoch 9/20
860277/860277 [==============================] - 45s - loss: 0.2888 - acc: 0.8789 - val_loss: 0.2598 - val_acc: 0.8906
Epoch 10/20
860277/860277 [==============================] - 45s - loss: 0.2827 - acc: 0.8820 - val_loss: 0.2473 - val_acc: 0.8993
Epoch 11/20
860277/860277 [==============================] - 46s - loss: 0.2772 - acc: 0.8852 - val_loss: 0.2435 - val_acc: 0.9018
Epoch 12/20
860277/860277 [==============================] - 46s - loss: 0.2729 - acc: 0.8880 - val_loss: 0.2470 - val_acc: 0.8986
Epoch 13/20
860277/860277 [==============================] - 45s - loss: 0.2684 - acc: 0.8904 - va

[{'class_name': 'Dense',
  'config': {'activation': 'sigmoid',
   'activity_regularizer': None,
   'batch_input_shape': (None, 198),
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'bias_regularizer': None,
   'dtype': 'float32',
   'kernel_constraint': None,
   'kernel_initializer': {'class_name': 'VarianceScaling',
    'config': {'distribution': 'uniform',
     'mode': 'fan_avg',
     'scale': 1.0,
     'seed': None}},
   'kernel_regularizer': None,
   'name': 'dense_53',
   'trainable': True,
   'units': 512,
   'use_bias': True}},
 {'class_name': 'Dropout',
  'config': {'name': 'dropout_27',
   'rate': 0.10466253116015645,
   'trainable': True}},
 {'class_name': 'Dense',
  'config': {'activation': 'tanh',
   'activity_regularizer': None,
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'bias_regularizer': None,
   'kernel_constraint': None,
   'kernel_initializer': {'class_name': 'VarianceScali

In [46]:
kfold = 1
skf = StratifiedKFold(n_splits=kfold, random_state=42)

ValueError: k-fold cross-validation requires at least one train/test split by setting n_splits=2 or more, got n_splits=1.

In [42]:
X = np.vstack((X_train,X_vali))

In [43]:
y = np.hstack((y_train,y_vali))

In [44]:
sub = pd.DataFrame()
sub['id'] = test_id
sub['target'] = np.zeros_like(test_id)

In [30]:
NUM_EPOCHS = 20
BATCH_SIZE = 100

sub = pd.DataFrame()
sub['id'] = test_id
sub['target'] = np.zeros_like(test_id)

for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print('[Fold %d/%d]' % (i + 1, kfold))
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    #class_weight = {1 : (len(y_train) - sum(y_train))/y_train.shape[0], 0: sum(y_train)/y_train.shape[0]}
    ES = EarlyStopping(monitor='val_loss', min_delta=0, patience=1, verbose=1, mode='auto')
    history = best_model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,validation_data=(X_valid,y_valid), callbacks=[ES,metrics])#, class_weight = class_weight
    print('gini metric:', metrics.ginis)
    print()
    print('[Fold %d/%d Prediciton:]' % (i + 1, kfold))
    # Predict on our test data
    p_test =best_model.predict_proba(X_test)[:,0]
    sub['target'] += p_test/kfold

[Fold 1/6]
Train on 955862 samples, validate on 191174 samples
Epoch 1/20
955862/955862 [==============================] - 32s - loss: 0.0942 - acc: 0.9658 - val_loss: 0.0990 - val_acc: 0.9668
Epoch 2/20
955862/955862 [==============================] - 31s - loss: 0.0920 - acc: 0.9667 - val_loss: 0.0949 - val_acc: 0.9688
Epoch 3/20
955862/955862 [==============================] - 31s - loss: 0.0901 - acc: 0.9675 - val_loss: 0.0948 - val_acc: 0.9689
Epoch 4/20
955862/955862 [==============================] - 32s - loss: 0.0886 - acc: 0.9680 - val_loss: 0.0945 - val_acc: 0.9687
Epoch 5/20
955862/955862 [==============================] - 32s - loss: 0.0867 - acc: 0.9685 - val_loss: 0.0945 - val_acc: 0.9691
Epoch 6/20
955862/955862 [==============================] - 32s - loss: 0.0850 - acc: 0.9693 - val_loss: 0.0942 - val_acc: 0.9695
Epoch 7/20
955862/955862 [==============================] - 32s - loss: 0.0835 - acc: 0.9697 - val_loss: 0.0937 - val_acc: 0.9693
Epoch 8/20
955862/955862 [=

In [31]:
import datetime as dt

In [49]:
ES = EarlyStopping(monitor='val_loss', min_delta=0, patience=1, verbose=1, mode='auto')
class_weight = {1 : (len(y_train) - sum(y_train))/y_train.shape[0], 0: sum(y_train)/y_train.shape[0]}
history = best_model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,validation_data=(X_valid,y_valid), callbacks=[ES,metrics], class_weight = class_weight)#, class_weight = class_weight
print('gini metric:', metrics.ginis)
print()
# Predict on our test data
sub['target'] = best_model.predict_proba(X_test)[:,0]

Train on 446409 samples, validate on 191172 samples
Epoch 1/20
446409/446409 [==============================] - 18s - loss: 0.0217 - acc: 0.8678 - val_loss: 0.1568 - val_acc: 0.9336
Epoch 2/20
446409/446409 [==============================] - 18s - loss: 0.0223 - acc: 0.8567 - val_loss: 0.1481 - val_acc: 0.9385
Epoch 3/20
446409/446409 [==============================] - 18s - loss: 0.0222 - acc: 0.8569 - val_loss: 0.1468 - val_acc: 0.9376
Epoch 4/20
446409/446409 [==============================] - 18s - loss: 0.0217 - acc: 0.8596 - val_loss: 0.1553 - val_acc: 0.9329
Epoch 5/20
446409/446409 [==============================] - 18s - loss: 0.0214 - acc: 0.8612 - val_loss: 0.1913 - val_acc: 0.9137
Epoch 00004: early stopping
gini metric: 0.994863668361

892480/892816 [============================>.] - ETA: 0s

In [50]:
filename = 'DL_model_'+str(dt.datetime.now()).replace(' ','_').replace(':','').replace('.','')
sub.id = sub.id.astype('Int32')
sub.to_csv('../output/'+filename+'.csv', index=False)
#test = np.load('../output/params.npy').item()